# How to handle connections to devices

## Connection options
By default, connections are handled automatically based on the information in the inventory.
Here is an example of *hosts.yaml* with simple and advanced connection details:

In [1]:
# NBVAL_IGNORE_OUTPUT
%run syntax_highlight_file.py handling_connections/example1/inventory/hosts.yaml yaml

```yaml
simple_host:
    hostname: simple-host.my-lab.com
    username: simple-user
    password: simple-password
    platform: cisco_ios

advanced_host:
    # standard parameters that will be re-used across different connections
    hostname: advanced-host.my-lab.com
    username: my_user
    password: my_password
    platform: linux
    connection_options:
        napalm:
          # standard parameters override upper level and will be used only for the napalm connection
          port: 443
          platform: eos
          extras:
              # non-standard library-specific options (check backend library docs)
              optional_args:
                  eos_autoComplete: true
        netmiko:
            platform: arista_eos
            extras:
                secret: "my-enable-password"
                global_delay_factor: 2
        netmiko_telnet:  # non-default connection name
            # terminal server details
            hostname: 1.2.3.4
            port: 3001
            platform: arista_eos_telnet
```

There are 5 reserved variable names that are used by Nornir for connection handling:
    
* **hostname**: hostname or IP address
* **username**
* **password**
* **port**: if not specified, the default port chosen by the backend library will be used
* **platform**: a connection plugin passes this value with a key supported by the backend library. For example, *netmiko* library uses *device_type*, so netmiko connection plugin will use platform value as device_type in netmiko. Refer to the backend library documentation to specify this parameter correctly.

You can specify these parameters on the host level, but you can also override default parameters on the connection level, which should be specified under **connection_options -> &lt;connection-name&gt;**

Let's consider connection options for *napalm*:  
**platform** option was overwritten and is now *eos* instead of *linux*, and the **port** overrides the default.

You can also specify non-standard library specific details under **extras**.
For example, during connection establishment *NAPALM* accepts an argument *optional_args*, while *netmiko* accepts an extra option *global_delay_factor* and *secret*.

The key name for the connection specific options is important and indicates **connection name**. In this example, we have *napalm*, *netmiko*, *netmiko_telnet*.

Let's consider the first two and we will get back to the third one in a moment.
As of this writing, Nornir has 3 connection-oriented plugins:

* NAPALM with the default connection name **napalm**  
* Netmiko with the default connection name **netmiko**  
* Paramiko with the default connection name **paramiko**  

The names can also be found inside each connection plugin class (`nornir/plugins/connections/` directory) with `name` attribute. These are default connection names which are needed to be used in the inventory if you would like to load these options automatically during the corresponding task execution.

This means that you can have only one connection established per a single backend library. Most of the times it is enough, but there are situations when you may need to have multiple connections established using the same backend library.  
For example, you would like to establish SSH and Telnet (via terminal server) connections using netmiko plugin. This is why you can specify arbitrary connection name as a key, e.g. **netmiko_telnet**.  

When you want to establish a custom connection name, you need to specify it somehow during task execution.  
This is done in `Nornir.run()` function with the **conn_name** argument.  
For example: `nr.run(task=..., conn_name="netmiko_telnet")` (see an example below)

Let's see how the connections are handled in the code, first automatically, then manually.
Here is the inventory used for the next examples:


In [2]:
# NBVAL_IGNORE_OUTPUT
%run syntax_highlight_file.py handling_connections/example2/inventory/hosts.yaml yaml

```yaml
rtr00:
    connection_options:
        napalm:
            platform: mock
            extras:
                optional_args:
                    path: handling_connections/example2/mocked_data
        napalm-secondary:  # non-default connection name
            platform: mock
            extras:
                optional_args:
                    path: handling_connections/example2/mocked_data
```

## Automatic connection handling

Networking tasks open connections automatically. Nornir can close them for you automatically as well, but for that you need to use Python context manager (`with` statement) as shown below. At the end of `with` block all open connections will be automatically closed even if an exception occurred.  
Using `with` statement is recommended.

#### Single connection

In [3]:
from nornir import InitNornir
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import napalm_get

In [4]:
with InitNornir(config_file="handling_connections/example2/config.yaml") as nr:
    rtr = nr.filter(name="rtr00")
    r = rtr.run(
        task=napalm_get,
        getters=["facts"]
    )  # connection will be opened here and stored with "napalm" key
    # it will be re-used if another task uses napalm
    print_result(r)
    print(f"The following connections are open before closing: {sorted(nr.inventory.hosts['rtr00'].connections.keys())}")
# all opened connections are closed here, where context manager block is done
print(f"The following connections are open after closing: {sorted(nr.inventory.hosts['rtr00'].connections.keys())}")

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
The following connections are open before closing: ['napalm']
The following connections are open after closing: []


You can also close connections explicitly using `Nornir.close_connections()`:

In [5]:
nr = InitNornir(config_file="handling_connections/example2/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=napalm_get,
    getters=["facts"]
)  # connection will be opened here and stored with "napalm" key
# it will be re-used if another tasks use napalm
print_result(r)
print(f"The following connections are open before closing: {sorted(nr.inventory.hosts['rtr00'].connections.keys())}")
nr.close_connections()  # all opened connections are closed here explicitly
print(f"The following connections are open after closing: {sorted(nr.inventory.hosts['rtr00'].connections.keys())}")

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
The following connections are open before closing: ['napalm']
The following connections are open after closing: []


#### Several connections using the same plugin

In [6]:
with InitNornir(config_file="handling_connections/example2/config.yaml") as nr:
    rtr = nr.filter(name="rtr00")
    r1 = rtr.run(
        task=napalm_get,
        getters=["facts"]
    )  # connection will be opened here and stored with "napalm" key
    # it will be re-used if another task uses connection "napalm"    
    print_result(r1)
    r2 = rtr.run(
        task=napalm_get,
        conn_name="napalm-secondary",  # conn_name is specified
        getters=["facts"]
    )  # connection will be opened here and stored with "secondary" key
    # it will be re-used if another task uses connection "secondary"
    print_result(r2)
    print(f"The following connections are open before closing: {sorted(nr.inventory.hosts['rtr00'].connections.keys())}")
# all opened connections are closed here
print(f"The following connections are open after closing: {sorted(nr.inventory.hosts['rtr00'].connections.keys())}")


napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** chang

## Manual connection handling 

In some circumstances, you may want to manage connections manually.  
The following functions can help you with that: 
[open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection), [close_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connection), [close_connections](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connections) and [Nornir.close_connections](../ref/api/nornir.rst#nornir.core.Nornir.close_connections).

If you choose to handle connections manually, it is also advised to use the argument `autoconnect=False` in the task run. Otherwise, nornir will try opening a connection if one isn't open already during task execution.

#### Manual handling of a single connection

In [7]:
def task_manages_connection_manually(task):
    task.host.open_connection("napalm", configuration=task.nornir.config)
    print(f"The following connections are open before running the task: {sorted(task.host.connections.keys())}")
    r = task.run(
        task=napalm_get,
        getters=["facts"],
        autoconnect=False,  # don't try opening the connection automatically
    )
    print(f"The following connections are open before closing: {sorted(task.host.connections.keys())}")
    task.host.close_connection("napalm")
    print(f"The following connections are open after closing: {sorted(task.host.connections.keys())}")
    
nr = InitNornir(config_file="handling_connections/example2/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=task_manages_connection_manually,
)
print_result(r)

The following connections are open before running the task: ['napalm']
The following connections are open before closing: ['napalm']
The following connections are open after closing: []
task_manages_connection_manually************************************************
* rtr00 ** changed : False *****************************************************
vvvv task_manages_connection_manually ** changed : False vvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_get ** changed : False --------------------------------------------- INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arist

#### Manual handling of multiple connections using the same plugin

In [8]:
def task_manages_multiple_connections_manually(task):
    task.host.open_connection("napalm", task.nornir.config)
    task.host.open_connection("napalm-secondary", task.nornir.config, plugin_name="napalm")
    print(f"The following connections are open before running tasks: {sorted(task.host.connections.keys())}")
    r1 = task.run(
        task=napalm_get,
        getters=["facts"]
    )
    r2 = task.run(
        task=napalm_get,
        conn_name="napalm-secondary",
        getters=["facts"]
    )
    task.host.close_connections()
    print(f"The following connections are open after closing: {sorted(task.host.connections.keys())}")
    
nr = InitNornir(config_file="handling_connections/example2/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=task_manages_multiple_connections_manually,
)
print_result(r)

The following connections are open before running tasks: ['napalm', 'napalm-secondary']
The following connections are open after closing: []
task_manages_multiple_connections_manually**************************************
* rtr00 ** changed : False *****************************************************
vvvv task_manages_multiple_connections_manually ** changed : False vvvvvvvvvvvvv INFO
---- napalm_get ** changed : False --------------------------------------------- INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
---- napalm_get ** changed : False -----